<a href="https://colab.research.google.com/github/julianVelandia/stable-diffusion-v1-5-finetuned-logos-white-background/blob/master/stable_diffusion_v1_5_finetuned_logos_white_background.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub diffusers transformers accelerate -q
!pip install huggingface_hub -q
!pip install accelerate
!pip install torchvision
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [2]:
import kagglehub
from huggingface_hub import HfApi, HfFolder, login
from diffusers import StableDiffusionPipeline, UNet2DConditionModel
from transformers import AutoTokenizer
import torch
import os
import json
from accelerate import Accelerator
from torch.optim import AdamW
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms
from huggingface_hub import login
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
from diffusers import StableDiffusionPipeline
from transformers import AutoTokenizer
from accelerate import Accelerator
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from PIL import Image


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
path = kagglehub.dataset_download("juliancamilovelandia/logos-with-white-background")

print("Path to dataset files:", path)

json_path = os.path.join(path, 'dataset_annotations_descriptions.json')
with open(json_path, 'r') as f:
    annotations = json.load(f)

print("Primeros elementos del JSON de anotaciones:", list(annotations.items())[:5])

100%|██████████| 15.0M/15.0M [00:01<00:00, 8.59MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/juliancamilovelandia/logos-with-white-background/versions/2
Primeros elementos del JSON de anotaciones: [('3320878793.jpg', 'El logotipo presenta una forma geométrica curva, reminiscente a una media luna, con un borde suave y redondeado. La figura tiene un contorno circular, aunque no perfecto, con pequeñas irregularidades que le dan un toque personal y artístico.\n\nLa parte superior del logotipo presenta una sección plana, mientras que la inferior es convexa, creando una sensación de profundidad y tridimensionalidad. El borde exterior está definido por una línea fina y negra, que a su vez rodea un espacio interior vacío.\n\nEn el centro del logotipo se encuentra un elemento circular, con un color cálido y dorado (aproximadamente #FFD700). Esta figura tiene una textura suave y lisa, con pequeños detalles que le dan relieve.\n\nLos colores utilizados en el logotipo son dos: el dorado mencionado anteriormente y un negro intenso (#00

In [4]:
login(token="...", add_to_git_credential=True)


Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
class LogosDataset(Dataset):
    def __init__(self, img_folder, annotations, transform=None):
        self.img_folder = img_folder
        self.annotations = annotations
        self.transform = transform or transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.ToTensor()
        ])
        self.image_files = list(annotations.keys())

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_folder, img_name)
        image = Image.open(img_path).convert("RGB")
        prompt = self.annotations[img_name]
        image = self.transform(image)
        return {"pixel_values": image, "prompt": prompt}

img_folder = os.path.join(path, 'processed_images')
dataset = LogosDataset(img_folder=img_folder, annotations=annotations)
train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


In [6]:
model_id = "sd-legacy/stable-diffusion-v1-5"
tokenizer = AutoTokenizer.from_pretrained(model_id, subfolder="tokenizer")
pipeline = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipeline = pipeline.to("cuda")

for param in pipeline.unet.parameters():
    param.requires_grad = True

accelerator = Accelerator( mixed_precision="fp16")
pipeline, train_dataloader = accelerator.prepare(pipeline, train_dataloader)

optimizer = AdamW(filter(lambda p: p.requires_grad, pipeline.unet.parameters()), lr=1e-5)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer/tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

scheduler/scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [7]:
epochs = 3
for epoch in range(epochs):
    for batch in tqdm(train_dataloader):
        with torch.no_grad():
            pixel_values = batch["pixel_values"].to("cuda").to(torch.float16)
            pixel_values = torch.cat([pixel_values, pixel_values[:, :1, :, :]], dim=1)

            prompts = batch["prompt"]

            inputs = tokenizer(prompts, padding=True, truncation=True, max_length=tokenizer.model_max_length, return_tensors="pt").to("cuda")
            text_embeddings = pipeline.text_encoder(input_ids=inputs.input_ids).last_hidden_state.to(torch.float16)

            noise = torch.randn_like(pixel_values).to(torch.float16)
            timesteps = torch.tensor([1], device="cuda", dtype=torch.float16)

        pixel_values.requires_grad = True
        text_embeddings.requires_grad = True
        noise_pred = pipeline.unet(pixel_values + noise, timesteps, encoder_hidden_states=text_embeddings).sample
        loss = ((noise_pred - noise) ** 2).mean()

        optimizer.zero_grad()
        accelerator.backward(loss)
        optimizer.step()

        pixel_values = pixel_values.cpu()
        text_embeddings = text_embeddings.cpu()
        noise = noise.cpu()
        noise_pred = noise_pred.cpu()

        del pixel_values, prompts, inputs, text_embeddings, noise, noise_pred, timesteps
        torch.cuda.empty_cache()
    print(f"Epoch {epoch + 1}/{epochs} completed. Loss: {loss.item()}")


  0%|          | 0/809 [00:00<?, ?it/s]

Epoch 1/3 completed. Loss: nan


  0%|          | 0/809 [00:00<?, ?it/s]

Epoch 2/3 completed. Loss: nan


  0%|          | 0/809 [00:00<?, ?it/s]

Epoch 3/3 completed. Loss: nan


In [17]:
from PIL import Image
import torch
import numpy as np

prompts = [
    "Logotipo minimalista de una empresa de tecnología, diseño en tonos gris y azul oscuro, sobre fondo claro",
    "Logotipo colorido para una empresa de alimentos, con colores brillantes y diseño vibrante sobre fondo claro",
    "Logotipo elegante para una firma legal, diseño clásico en negro y dorado, en un fondo claro",
    "Logotipo moderno con iconos geométricos en tonos metálicos, con formas abstractas sobre fondo claro"
]

pipeline.safety_checker = None

for i, prompt in enumerate(prompts):
    with torch.no_grad():
        result = pipeline(prompt, num_inference_steps=50, guidance_scale=7.5)
        image = result.images[0]

    image_array = np.array(image)
    print(f"Prompt: {prompt}")

    image_array = np.clip(image_array * 255, 0, 255).astype(np.uint8)
    image = Image.fromarray(image_array)

    image.save(f"generated_logo_{i+1}.png")


  0%|          | 0/50 [00:00<?, ?it/s]

Prompt: Logotipo minimalista de una empresa de tecnología, diseño en tonos gris y azul oscuro, sobre fondo claro


/usr/local/lib/python3.10/dist-packages/diffusers/image_processor.py:111: RuntimeWarning: invalid value encountered in cast
  images = (images * 255).round().astype("uint8")


  0%|          | 0/50 [00:00<?, ?it/s]

Prompt: Logotipo colorido para una empresa de alimentos, con colores brillantes y diseño vibrante sobre fondo claro


  0%|          | 0/50 [00:00<?, ?it/s]

Prompt: Logotipo elegante para una firma legal, diseño clásico en negro y dorado, en un fondo claro


  0%|          | 0/50 [00:00<?, ?it/s]

Prompt: Logotipo moderno con iconos geométricos en tonos metálicos, con formas abstractas sobre fondo claro


In [15]:
from huggingface_hub import HfApi, login

login(token="...")

repo_id = "JulianVelandia/stable-diffusion-v1-5-finetuned_logos_white_background"
api = HfApi()

api.create_repo(repo_id=repo_id, repo_type="model", exist_ok=True)

save_path = "./finetuned_logos_white_background"
pipeline.save_pretrained(save_path)

api.upload_folder(folder_path=save_path, path_in_repo="", repo_id=repo_id, repo_type="model")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/JulianVelandia/stable-diffusion-v1-5-finetuned_logos_white_background/commit/a4c4798059243901c2070595a13b5b0953392c62', commit_message='Upload folder using huggingface_hub', commit_description='', oid='a4c4798059243901c2070595a13b5b0953392c62', pr_url=None, pr_revision=None, pr_num=None)